In [48]:
import numpy as np
import pandas as pd
import os

## 1. 데이터 불러오기

In [26]:
raw_item_data = pd.read_excel("/opt/ml/input/data/raw_codishop/view/item/item.xlsx")
df = raw_item_data[["id", "big_class", "mid_class"]]
unique_big_class = list(df["big_class"].unique())
unique_mid_class = list(df["mid_class"].unique())

## 2. 데이터 확인하기

In [27]:
bc_nan_count = df["big_class"].isnull().sum()
mc_nan_count = df["mid_class"].isnull().sum()
print(f"대분류 결측치 개수 : {bc_nan_count}")
print(f"중분류 결측치 개수 : {mc_nan_count}")

대분류 결측치 개수 : 0
중분류 결측치 개수 : 0


In [28]:
for big_class in unique_big_class :
    print(f"대분류 {big_class} 안에 있는 중분류는 : ")
    print(df[df["big_class"]==big_class]["mid_class"].unique())
    print()

대분류 아우터 안에 있는 중분류는 : 
['트러커 재킷' '슈트/블레이저 재킷' '후드 집업' '숏패딩/숏헤비 아우터' '겨울 더블 코트' '겨울 싱글 코트'
 '블루종/MA-1' '사파리/헌팅 재킷' '레더/라이더스 재킷' '베스트' '카디건' '겨울 기타 코트' '플리스/뽀글이'
 '나일론/코치 재킷' '기타 아우터' '트레이닝 재킷' '환절기 코트' '무스탕/퍼' '스타디움 재킷' '롱패딩/롱헤비 아우터'
 '아노락 재킷' '패딩 베스트']

대분류 상의 안에 있는 중분류는 : 
['맨투맨/스웨트셔츠' '셔츠/블라우스' '반소매 티셔츠' '후드 티셔츠' '긴소매 티셔츠' '피케/카라 티셔츠' '니트/스웨터'
 '민소매 티셔츠']

대분류 바지 안에 있는 중분류는 : 
['데님 팬츠' '트레이닝/조거 팬츠' '숏 팬츠' '슈트 팬츠/슬랙스' '코튼 팬츠' '기타 바지']

대분류 가방 안에 있는 중분류는 : 
['백팩' '토트백' '메신저/크로스 백' '웨이스트 백' '숄더백' '파우치 백' '브리프케이스' '클러치 백']

대분류 스니커즈 안에 있는 중분류는 : 
['캔버스/단화' '패션스니커즈화' '기타 스니커즈' '농구화']

대분류 액세서리 안에 있는 중분류는 : 
['넥타이' '벨트' '머플러' '기타 액세서리' '스카프/반다나' '장갑' '마스크']

대분류 신발 안에 있는 중분류는 : 
['로퍼' '블로퍼' '부츠' '구두' '샌들' '슬리퍼' '모카신/보트 슈즈']

대분류 모자 안에 있는 중분류는 : 
['캡/야구 모자' '헌팅캡/베레모' '비니' '버킷/사파리햇' '트루퍼']

대분류 선글라스/안경테 안에 있는 중분류는 : 
['안경' '선글라스']

대분류 양말/레그웨어 안에 있는 중분류는 : 
['양말']

대분류 주얼리 안에 있는 중분류는 : 
['팔찌' '반지' '목걸이/펜던트' '발찌']

대분류 스포츠/용품 안에 있는 중분류는 : 
['스포츠가방' '스포츠신발' '스포츠잡화']

대분류 시계 안에 있는 중분류는 : 
['디지털'

## 3. 데이터 전처리

수정사항
형식 : (기존 대분류 -> 기존 중분류) >> (수정된 대분류 -> 수정된 중분류)

1. (스포츠/용품 -> 스포츠신발) >> (신발 -> 스포츠신발)
2. (스니커즈 -> 캔버스/단화, 패션스니커즈화, 기타 스니커즈, 농구화) >> (신발 -> 캔버스/단화, 패션스니커즈화, 기타 스니커즈, 농구화)
3. [대분류 선글라스/안경테, 양말/레그웨어, 주얼리, 시계, 생활/취미/예술, 스포츠/용품]에 있는 중분류들은 액세서리로 넣는다.
4. 스포츠가방, 백팩, 크로스백 -> 대분류 가방으로 설정


In [29]:
for mid_class in ['캔버스/단화', '패션스니커즈화', '기타 스니커즈', '농구화'] :
    df.loc[df["mid_class"]==mid_class, "big_class"] = "신발"

for mid_class in ['안경', '선글라스', '양말', '팔찌', '반지', '목걸이/펜던트', '발찌', '스포츠잡화', '디지털', '쿼츠 아날로그', '오토매틱 아날로그', '카메라/카메라용품', '우산'] :
    df.loc[df["mid_class"]==mid_class, "big_class"] = "액세서리"

df.loc[df["mid_class"]=="스포츠신발", "big_class"] = "신발"

for mid_class in ["스포츠가방",'백팩', '크로스백'] :
    df.loc[df["mid_class"]==mid_class, "big_class"] = "가방"

## 4. 액세서리, 속옷 제거 후 엑셀 파일로 반환

In [47]:
# 제거 전 대분류
raw_item_data["big_class"].unique()

array(['아우터', '상의', '바지', '가방', '신발', '모자'], dtype=object)

In [35]:
# 전처리한 대분류, 중분류 원 데이터에 반영
raw_item_data["big_class"] = df["big_class"]
raw_item_data["mid_class"] = df["mid_class"]

# 액세서리, 속옷 대분류 제거 
accessory_index = raw_item_data[raw_item_data["big_class"]=="액세서리"].index
underwear_index = raw_item_data[raw_item_data["big_class"]=="속옷"].index
raw_item_data.drop(accessory_index, inplace=True)
raw_item_data.drop(underwear_index, inplace=True)

# 출력 엑셀의 형식을 원래 데이터의 형식과 동일하게 맞춰주기 위한 부분
raw_item_data["id"] = raw_item_data["id"].apply(str)

In [44]:
# 제거 후 대분류 
raw_item_data["big_class"].unique()

In [49]:
# 엑셀로 반환 

raw_item_data.to_excel("/opt/ml/input/data/asset_codishop/view/item/item.xlsx", index=False)